In [ ]:
class RSVPTracker:
    def __init__(self):
        self.tracking = set()
    def rsvp(self, user_id, event_id):
        self.tracking.add((user_id, event_id))
    def cancel(self, user_id, event_id):
        self.tracking.discard((user_id, event_id))
    def has_rsvped(self, user_id, event_id):
        return (user_id, event_id) in self.tracking
    def rsvp_count(self, event_id):
        count = 0
        for items in self.tracking:
            if event_id == items[1]:
                count += 1
        return count

tracker = RSVPTracker()
tracker.rsvp("1", "101a")
tracker.rsvp("2", "101a")
tracker.rsvp("3", "102")
print(tracker.has_rsvped("1", "101a"))  # True
print(tracker.has_rsvped("2", "101a"))  # True
print(tracker.has_rsvped("3", "102"))  # True
print(tracker.rsvp_count("101a"))  # 2
tracker.cancel("1", "101a")

#add() complexity: O(1) average case
#discard() complexity: O(1) average case
#in operator complexity: O(1) average case
#iterating through the set complexity: O(n) where n is the number of elements in the set


# rsvp(user_id, event_id) — Time? Space? O(1) time complexity for adding an RSVP, O(1) space complexity for storing the RSVP in the set.

# cancel(user_id, event_id) — Time? Space? O(1) time complexity for removing an RSVP, O(1) space complexity for storing the RSVP in the set.

# has_rsvped(user_id, event_id) — Time? Space? O(1) time complexity for checking if an RSVP exists, O(1) space complexity for storing the RSVP in the set.

# rsvp_count(event_id) — Time? Space? O(n) time complexity for counting RSVPs for a specific event, where n is the total number of RSVPs in the set. O(1) space complexity for storing the count variable.

# Overall space used by the class as the number of RSVPs grows — O(n) where n is the number of RSVPs in the set.

---
---
---
Great — we’re done with Phase 1.

## Phase 2: The Scalability Pivot (25 mins)

**How would this work for 100 million users?**

Imagine your `RSVPTracker` is now a real production feature:

* Users RSVP/cancel for events
* You must support: `rsvp`, `cancel`, `has_rsvped`, `rsvp_count`
* It needs to work at **massive scale** (100M users), across multiple servers (not one Python process)

### Your rules for answering (mandatory)

Walk me through using this exact **5-step methodology**, with headers:

1. **Clarifying Questions**
2. **NALSD (Back-of-the-envelope math)**
3. **Component Interaction** (high-level architecture + how requests flow)
4. **Logical Patterns** (sharding / queues / caching / indexing — whatever you choose)
5. **Trade-offs** (include **CAP theorem** discussion)

Start with **Step 1: Clarifying Questions** — ask me your questions first.


1. What is the existing or as it set-up? 2. What are the main choke or pain points? 3. Are there any SLO, SLAs in place? if yes, how are they doing? 4. Are there any cost constraints?

Here’s the context (as if I’m the hiring team / product owner answering your clarifying questions):

### 1) What is the existing setup?

* Today it’s basically what you coded: **one service instance** keeping RSVPs **in memory** (no database).
* It restarts sometimes (deploys/auto-healing), so data can be lost.
* You can assume there’s an API layer like:

  * `POST /events/{event_id}/rsvp`
  * `DELETE /events/{event_id}/rsvp`
  * `GET /events/{event_id}/rsvp_count`
  * `GET /events/{event_id}/users/{user_id}` (has_rsvped)

### 2) Main choke/pain points?

* **Memory limit** (can’t store enough RSVP pairs).
* **Single point of failure** (one instance dies → feature down).
* **No durability** (restart = data loss).
* **Hot events**: a small % of events get *huge* traffic spikes (think concert tickets).

### 3) Any SLOs / SLAs?

* Availability: **99.9%** monthly for RSVP feature.
* Latency targets (p95):

  * `has_rsvped`: **< 50 ms**
  * `rsvp/cancel`: **< 150 ms**
  * `rsvp_count`: **< 100 ms**
* Consistency expectations:

  * For a single user: after they RSVP/cancel, **their own `has_rsvped` should reflect it quickly** (within ~1s).
  * `rsvp_count` can be **slightly stale** (up to ~5 seconds) during traffic spikes.

### 4) Cost constraints?

* Reasonable cost sensitivity: “don’t build a super expensive system for a simple feature.”
* Prefer **managed services** where it makes sense.
* Start **single-region**; have a **disaster recovery plan**, but not active-active multi-region on day 1.

Now move to **Step 2: NALSD (back-of-the-envelope math)**.
Give me your assumptions + rough numbers for:

* total RSVPs stored
* peak reads/writes per second
* storage size estimate
* hottest-event spike estimate
